In [1]:
%matplotlib inline
import codecs
from collections import Counter
import random 
import numpy as np
from numpy.random import permutation, shuffle, rand

from scipy.optimize import minimize

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn import svm

from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

In [2]:
sentences = list()
labels = list()

labels_dist = set()

dataset = dict()
#We will release training and testing data for the following Arabic dialects: 
# Egyptian, Gulf, Levantine, and North-African, and Modern Standard Arabic (MSA)

with codecs.open('task2-train.txt') as training:
    LAV = list()
    MSA = list()
    EGY = list()
    GLF = list()
    NOR = list()
    for i, line in enumerate(training):
        sentence_label = line.strip().split('\t')
        # sentences.append(sentence_label[0])
        # labels.append(sentence_label[2])
        
        if sentence_label[2] == 'LAV':
            LAV.append(sentence_label[0])
        elif sentence_label[2] == 'MSA':
            MSA.append(sentence_label[0])
        elif sentence_label[2] == 'EGY':
            EGY.append(sentence_label[0])
        elif sentence_label[2] == 'GLF':
            GLF.append(sentence_label[0])
        elif sentence_label[2] == 'NOR':
            NOR.append(sentence_label[0])
        else:
            print(sentence_label[0])
    else:
        print 'sentence count:', len(sentences)
        print set(labels)
        dataset['LAV'] = LAV
        dataset['MSA'] = MSA
        dataset['EGY'] = EGY
        dataset['GLF'] = GLF
        dataset['NOR'] = NOR
        LAV = list()
        MSA = list()
        EGY = list()
        GLF = list()
        NOR = list()

target_names = dataset.keys()

sentence count: 0
set([])


In [3]:
len(dataset['LAV'])

1758

In [18]:
average_word_len = 0;
num_words = 0;
for s in dataset['LAV'] + dataset['MSA'] + dataset['EGY'] + dataset['GLF'] + dataset['NOR']:
    num_words += len(s.split(' '))
    average_word_len += sum([len(w) for w in s.split(' ')])
print float(average_word_len)/num_words

4.43379487001


In [4]:
def divide_dataset(dataset ,CV=True, train_perc=60 , CV_perc=20, test_perc=20):
    if train_perc + CV_perc + test_perc != 100:
        print 'the sum of percs is not 100'
        return
    samples_train = dict()
    samples_cv = dict()
    samples_test = dict()
    
    for dialect, sentences in dataset.items():
        samples = permutation(sentences)
        train_len = int(np.ceil(len(samples)*(train_perc/100.0)))
        samples_train[dialect] = sentences[:train_len]
        cv_len = 0
        if CV:
            cvp = CV_perc/(100.0-60)
            cv_len = int(np.ceil((len(samples)-train_len) * cvp))
            samples_cv[dialect] = sentences[train_len:train_len+cv_len]
            samples_test[dialect] = sentences[train_len+cv_len:]
        else:
            samples_cv[dialect] = list()
            samples_test[dialect] = sentences[train_len:]
    else:
        return samples_train, samples_cv, samples_test
            

train_set, cv_set, test_set = divide_dataset(dataset, CV=False, train_perc=90 ,CV_perc=0, test_perc=10)


t,c,ts = 0,0,0
for dial in ['LAV', 'MSA', 'EGY', 'GLF', 'NOR']:
    t += len(train_set[dial])
    c += len(cv_set[dial])
    ts+= len(test_set[dial])
    print dial, 'training dataset: ', len(train_set[dial]), ', cross-validation set: ', \
    len(cv_set[dial]),', test:', len(test_set[dial])
    
else:
    print 70*'-'
    print 'Total  ...  Training: ', t, ', cross-validation data', c, ', test: ', ts

dataset_train = train_set['LAV']+train_set['MSA']+train_set['EGY']+train_set['GLF']+train_set['NOR']
dataset_cv = cv_set['LAV']+cv_set['MSA']+cv_set['EGY']+cv_set['GLF']+cv_set['NOR']
dataset_test = test_set['LAV']+test_set['MSA']+test_set['EGY']+test_set['GLF']+test_set['NOR']


label_train = ['LAV' for x in train_set['LAV']] + ['MSA' for x in train_set['MSA']] +\
['EGY' for x in train_set['EGY']] + ['GLF' for x in train_set['GLF']]+['NOR' for x in train_set['NOR']]

label_cv = ['LAV' for x in cv_set['LAV']] + ['MSA' for x in cv_set['MSA']] +\
['EGY' for x in cv_set['EGY']] + ['GLF' for x in cv_set['GLF']]+['NOR' for x in cv_set['NOR']]

label_test = ['LAV' for x in test_set['LAV']] + ['MSA' for x in test_set['MSA']] +\
['EGY' for x in test_set['EGY']] + ['GLF' for x in test_set['GLF']]+['NOR' for x in test_set['NOR']]

train_set, cv_set, test_set = 0,0,0
#print len(label_train),len(label_cv),len(label_test)

LAV training dataset:  1583 , cross-validation set:  0 , test: 175
MSA training dataset:  900 , cross-validation set:  0 , test: 99
EGY training dataset:  1421 , cross-validation set:  0 , test: 157
GLF training dataset:  1505 , cross-validation set:  0 , test: 167
NOR training dataset:  1451 , cross-validation set:  0 , test: 161
----------------------------------------------------------------------
Total  ...  Training:  6860 , cross-validation data 0 , test:  759


In [5]:
train_zipped = zip(dataset_train, label_train)
random.shuffle(train_zipped)
dataset_train, label_train = zip(*train_zipped)

if dataset_cv:
    cv_zipped = zip(dataset_cv, label_cv)
    random.shuffle(cv_zipped)
    dataset_cv, label_cv = zip(*cv_zipped)

if dataset_test:
    test_zipped = zip(dataset_test, label_test)
    random.shuffle(test_zipped)
    dataset_test, label_test = zip(*test_zipped)

### word count + Multinomial Naive Bayes 

In [6]:
count_vect = CountVectorizer() 
X_train = count_vect.fit_transform(dataset_train)
X_test = count_vect.transform(dataset_test)

mnb = MultinomialNB()
mnb.fit(X_train, label_train)

train_pred = mnb.predict(X_train)
test_pred = mnb.predict(X_test)

#count_vect.vocabulary_.get(u'la')
print 'Training Acc: ',np.around(np.mean(train_pred == label_train)*100,2), '%'
print 'Testing Acc: ',np.around(np.mean(test_pred == label_test)*100,2), '%'

Training Acc:  87.1 %
Testing Acc:  57.58 %


### TF + Multinomial Naive Bayes 

In [7]:
# To avoid these potential discrepancies it suffices to divide the number of occurrences of each word in a document 
# by the total number of words in the document: these new features are called tf for Term Frequencies.
tf_transformer = TfidfTransformer(use_idf=False) #.fit(X_train_counts)
X_train = tf_transformer.fit_transform(X_train)
X_test = tf_transformer.transform(X_test)

mnb = MultinomialNB()
mnb.fit(X_train, label_train)

train_pred = mnb.predict(X_train)
test_pred = mnb.predict(X_test)

#count_vect.vocabulary_.get(u'la')
print 'Training Acc: ',np.around(np.mean(train_pred == label_train)*100,2), '%'
print 'Testing Acc: ',np.around(np.mean(test_pred == label_test)*100,2), '%'

Training Acc:  72.86 %
Testing Acc:  44.14 %


### tfidf + Multinomial Naive Bayes 

In [8]:
tfidf_vect = TfidfVectorizer()
X_train = tfidf_vect.fit_transform(dataset_train)
X_test = tfidf_vect.transform(dataset_test)

mnb = MultinomialNB()
mnb.fit(X_train, label_train)
# X_new_counts = count_vect.transform(docs_new)
# X_new_tfidf = tfidf_transformer.transform(X_new_counts)


train_pred = mnb.predict(X_train)
test_pred = mnb.predict(X_test)
#actual = np.array(label_train, dtype='S3')
print 'Training Acc: ',np.around(np.mean(train_pred == label_train)*100,2), '%'
print 'Testing Acc: ',np.around(np.mean(test_pred == label_test)*100,2), '%'

Training Acc:  86.46 %
Testing Acc:  49.14 %


### tfidf + Logistic Regression

In [9]:

t = CountVectorizer() 
X_train = count_vect.fit_transform(dataset_train)
X_test = count_vect.transform(dataset_test)

logreg = LogisticRegression(C=1)
logreg.fit(X_train, label_train)
y_pred = logreg.predict(X_test)                  # .score(X_train, label_train)


print 'Training Acc: ',np.around(logreg.score(X_train, label_train)*100,2), '%'
print 'Testing Acc: ',np.around(logreg.score(X_test, label_test)*100,2), '%'

Training Acc:  96.38 %
Testing Acc:  56.26 %


### tfidf + Stochastic Gradient Descent (SGD)

In [10]:
clf = SGDClassifier()

# count_vect = CountVectorizer() 
# X_train = count_vect.fit_transform(dataset_train)
# X_test = count_vect.transform(dataset_test)

tfidf_vect = TfidfVectorizer()
X_train = tfidf_vect.fit_transform(dataset_train)
X_test = tfidf_vect.transform(dataset_test)

clf.fit(X_train, label_train)
train_pred = clf.predict(X_train) 
y_pred = clf.predict(X_test) 


print 'Training Acc: ',np.around(np.mean(train_pred == label_train)*100,2), '%'
print 'Testing Acc: ',np.around(np.mean(y_pred == label_test)*100,2), '%'

Training Acc:  96.43 %
Testing Acc:  59.82 %


### TF-IDF  + SVM

In [11]:
tfidf_vect = TfidfVectorizer()
X_train = tfidf_vect.fit_transform(dataset_train)
X_test = tfidf_vect.transform(dataset_test)

my_C = 100

# clf = svm.SVC()
# clf.fit(X_train, label_train)
# train_pred = clf.predict(X_train) 
# y_pred = clf.predict(X_test) 

clf_linear = svm.LinearSVC(C= 0.3)
clf_linear.fit(X_train, label_train)
train_pred = clf_linear.predict(X_train) 
y_pred = clf_linear.predict(X_test) 

print 'Training Acc: ',np.around(np.mean(train_pred == label_train)*100,2), '%'
print 'Testing Acc: ',np.around(np.mean(y_pred == label_test)*100,2), '%'

Training Acc:  96.02 %
Testing Acc:  59.29 %


### TF-IDF  + Neural Network
https://github.com/timshenkao/StringKernelSVM

In [ ]:
# dataset_train = ['We present a study on sentence-level Arabic Dialect Identification using the newly developed Multidialectal Parallel Corpus of Arabic (MPCA) – the first experiments on such data.', 'Using a set of surface features based on characters and words, we conduct three experiments with a linear Support Vector Machine classifier and a meta-classifier using stacked generalization – a method not previously applied for this task.', 'We first conduct a 6-way multi-dialect classification task in the first experiment, achieving 74% accuracy against a random baseline of 16.7% and demonstrating that meta-classifiers can large performance increases over single classifiers.', 'The second experiment investigates pairwise binary dialect classification within the corpus, yielding results as high as 94%, but also highlighting poorer results between closely related dialects such as Palestinian and Jordanian (76%).', 'Our final experiment conducts cross-corpus evaluation on the widely used Arabic Online Commentary (AOC) dataset and demonstrates that despite differing greatly in size and content, models trained with the MPCA generalize to the AOC, and vice versa.', 'Using only 2,000 sentences from the MPCA, we classify over 26k sentences from the radically different AOC dataset with 74% accuracy.', 
# 'We also use this data to classify a new dataset of MSA and Egyptian Arabic tweets with 97% accuracy.', 'We find that character n-grams are a very informative feature for this task, in both within-and cross-corpus settings.', 'Contrary to previous results, they outperform word n-grams in several experiments here.', 'Several directions for future work are outlined.']

# dataset_test = ['The  Arabic  language,  the  official  language  of  more  than 20 countries,  is  comprised  of  many  regional  dialects  with the Modern Standard Arabic (MSA) variety having the role of a common dialect across the Arabic-speaking population.', 'Arabic  is  a  morphologically  sophisticated  language  withmany  morphemes  that  can  appear  as  prefixes,  suffixes  oreven  circumfixes.',  'These  mark  grammatical  information  including  case,  number,  gender,  and  definiteness,  amongst others.',  'This  leads  to  a  sophisticated  morphotactic  system.', 
# 'Its orthography is very different to English with right-to-left text  that  uses  connective  letters.', 'Moreover,  this  is  further complicated  due  to  the  presence  of  word  elongation,  common ligatures, zero-width diacritics and allographic variants resulting  in  a  degree  of  orthographic  ambiguity.', ' All  ofthese properties pose a challenge for NLP [1].',
# 'Arabic Dialect Identification using a Parallel Multidialectal Corpus.']

In [ ]:
cm = confusion_matrix(label_test, y_pred, labels=[ 'MSA', 'LAV', 'EGY', 'GLF', 'NOR'])
cm

In [ ]:
cm.sum(axis=1)

In [ ]:
def plot_confusion_matrix(cm, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    #tick_marks = np.arange(len(iris.target_names))
    #plt.xticks(tick_marks, iris.target_names, rotation=45)
    #plt.yticks(tick_marks, iris.target_names)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')


# Compute confusion matrix
#cm = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)
print('Confusion matrix, without normalization')
print(cm)
plt.figure()
plot_confusion_matrix(cm)

# Normalize the confusion matrix by row (i.e by the number of samples
# in each class)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
print('Normalized confusion matrix')
print(cm_normalized)
plt.figure()
plot_confusion_matrix(cm_normalized, title='Normalized confusion matrix')

plt.show()


<pre>
```

| This | is |
|------|------|
|   a  | table|

```
</pre>